In [ ]:
import pandas as pd

## 2. Data Loading

#### 2.1 Data Collection

In [100]:
data = pd.read_csv('data/animal-shelter-intakes-and-outcomes.csv')

animal_data = data[['Animal Type', "DOB", 'Primary Color', 'Secondary Color', 'Sex', 'Intake Date', 'Intake Condition', 'Intake Type', 'Outcome Date',
                    'Outcome Type']].copy()

display(animal_data.head())

,Animal Type,DOB,Primary Color,Secondary Color,Sex,Intake Date,Intake Condition,Intake Type,Outcome Date,Outcome Type
0,DOG,2013-02-21,WHITE,NaN,Female,2023-02-20,ILL MILD,STRAY,2023-02-26,EUTHANASIA
1,REPTILE,NaN,BROWN,GREEN,Unknown,2023-10-03,NORMAL,STRAY,2023-10-03,RESCUE
2,BIRD,NaN,GREEN,RED,Unknown,2020-01-01,INJURED SEVERE,WILDLIFE,2020-01-01,EUTHANASIA
3,BIRD,NaN,WHITE,GRAY,Unknown,2020-02-02,ILL SEVERE,WILDLIFE,2020-02-02,TRANSFER
4,CAT,2014-12-18,BLACK,WHITE,Female,2018-12-18,INJURED SEVERE,STRAY,2019-01-13,RESCUE


#### 2.1.1 Data Overview

In [101]:
print(animal_data.shape)
print(animal_data.info())
print(animal_data.isnull().sum())

(30475, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30475 entries, 0 to 30474
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Animal Type       30475 non-null  object
 1   DOB               26717 non-null  object
 2   Primary Color     30475 non-null  object
 3   Secondary Color   14587 non-null  object
 4   Sex               30475 non-null  object
 5   Intake Date       30475 non-null  object
 6   Intake Condition  30475 non-null  object
 7   Intake Type       30475 non-null  object
 8   Outcome Date      30284 non-null  object
 9   Outcome Type      30276 non-null  object
dtypes: object(10)
memory usage: 2.3+ MB
None
Animal Type             0
DOB                  3758
Primary Color           0
Secondary Color     15888
Sex                     0
Intake Date             0
Intake Condition        0
Intake Type             0
Outcome Date          191
Outcome Type          199
dtype: int64


## 2.2 Data Cleaning

#### 2.2.1 Standardize Column Names

In [102]:
animal_data = animal_data.rename(
    columns={
            "Animal Type": "Animal_Type",
            "DOB" : "DOB",
            "Primary Color" : "Primary_Color",
            "Secondary Color" : "Secondary_Color",
            "Sex" : "Sex",
            "Intake Date" : "Intake_Date",
            "Intake Condition": "Intake_Condition",
            "Intake Type" : "Intake_Type",
            "Outcome Date" : "Outcome_Date",
            "Outcome Type" : "Outcome_Type"
    }
)

#### 2.2.2 Data Type Conversion

In [103]:
animal_data = animal_data.astype(
    {"Animal_Type" : "object",
     "DOB" : "datetime64[ns]",
     "Primary_Color" : "category",
     "Sex" : "category",
     "Intake_Date" : "datetime64[ns]",
     "Intake_Condition" : "category",
     "Intake_Type" : "category",
     "Outcome_Date" : "datetime64[ns]",
     }
)

animal_data["DOB"] = pd.to_datetime(animal_data["DOB"], errors='coerce')
animal_data["Intake_Date"] = pd.to_datetime(animal_data["Intake_Date"], errors='coerce')
animal_data["Outcome_Date"] = pd.to_datetime(animal_data["Outcome_Date"], errors='coerce')

#### 2.2.3 Fill Null Secondary Color

In [104]:
animal_data['Secondary_Color'] = animal_data['Secondary_Color'].fillna("None")
animal_data['Secondary_Color'] = animal_data['Secondary_Color'].astype('category')

#### 2.2.4 Fill Null Outcome Type

In [105]:
animal_data['Outcome_Type'] = animal_data['Outcome_Type'].fillna("Unknown")
animal_data['Outcome_Type'] = animal_data['Outcome_Type'].astype('category')

#### 2.2.5 Calculate Intake Age

In [106]:
animal_data['Intake_Age'] = animal_data['Intake_Date'] - animal_data['DOB']

#### 2.2.6 Calculate Outcome Age

In [107]:
animal_data['Outcome_Age'] = animal_data['Outcome_Date'] - animal_data['DOB']

#### 2.2.7 Calculate Days in Shelter

In [108]:
animal_data['Time_in_Shelter_Days'] = animal_data['Outcome_Date'] - animal_data['Intake_Date']

#### 2.2.8 Remove Outliers

In [109]:
animal_data = animal_data[
    (animal_data['Intake_Age'] > pd.Timedelta('0')) &
    (animal_data['Intake_Age'] < pd.Timedelta('18000 days 00:00:00')) &
    (animal_data['Outcome_Age'] < pd.Timedelta('18000 days 00:00:00'))
]
animal_data = animal_data.reset_index(drop=True)

#### 2.2.9 Standardize Intake Age

In [110]:
animal_data['Intake_Age'] = animal_data['Intake_Age'].astype(str)
animal_data['Intake_Age_Days'] = animal_data['Intake_Age'].str.replace(' days', '', regex=False)

#### 2.2.10 Unknown Breed Feature

In [111]:
animal_data['Breed'] = 'Unknown'

#### 2.2.11 Insert Shelter Name

In [112]:
animal_data['Shelter_Name'] = 'Long Beach California Animal Shelter'

## 2.3 Dataset Inspection & Cleaning

In [113]:
animal_data.shape

(26047, 16)

#### 2.3.1 Drop Duplicates

In [114]:
display(animal_data.duplicated().sum())
animal_data = animal_data.drop_duplicates()

np.int64(1329)

#### 2.3.2 Missing Values

In [115]:
animal_data.isna().sum()

Animal_Type             0
DOB                     0
Primary_Color           0
Secondary_Color         0
Sex                     0
Intake_Date             0
Intake_Condition        0
Intake_Type             0
Outcome_Date            0
Outcome_Type            0
Intake_Age              0
Outcome_Age             0
Time_in_Shelter_Days    0
Intake_Age_Days         0
Breed                   0
Shelter_Name            0
dtype: int64

## 3. Export Long Beach Animal Shelter Dataset

In [116]:
animal_data.to_csv('Long_Beach_Animal_Data.csv', index=False)
animal_data.columns

Index(['Animal_Type', 'DOB', 'Primary_Color', 'Secondary_Color', 'Sex',
       'Intake_Date', 'Intake_Condition', 'Intake_Type', 'Outcome_Date',
       'Outcome_Type', 'Intake_Age', 'Outcome_Age', 'Time_in_Shelter_Days',
       'Intake_Age_Days', 'Breed', 'Shelter_Name'],
      dtype='object')